DEFINE SOME USEFUL FUNCTION AND IMPORTING LIBRARIES

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
#import pingouin as pg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer
from utils import *

FIND FEATURE IMPORTANCE

In [22]:
path_dev = "./data/development.csv"
##  Importing Dataset
df = pd.read_csv(path_dev)

## Removing non relevant feature
df.drop(['url','id','timedelta'], axis = 1, inplace = True)

##  One-Hot Encoding
df = pd.get_dummies(df, columns=['data_channel', 'weekday'])

##  Studying Correlation between features in order to remove features highly correlated  using spearman method because the problem is non linear
top_10_values = {}
dictionary = {}
correlation_matrix = df.corr(method = 'spearman')
for el in df.columns:
  top_10_values[el] = correlation_matrix[el].abs().nlargest(35)[1:]
for el in df.columns:
  for i in range(0,10):
    if(top_10_values[el][i] > 0.7):
      if el in dictionary:
        dictionary[el].append(top_10_values[el].index[i])
      else:
        dictionary[el] = [top_10_values[el].index[i]]

## Feature highly correlated and feature important to our target value
print(dictionary)
print(top_10_values["shares"])

{'n_tokens_content': ['n_non_stop_words', 'n_unique_tokens'], 'n_unique_tokens': ['n_non_stop_unique_tokens', 'n_tokens_content', 'n_non_stop_words'], 'n_non_stop_words': ['n_tokens_content', 'n_unique_tokens'], 'n_non_stop_unique_tokens': ['n_unique_tokens'], 'kw_min_min': ['kw_max_max'], 'kw_max_min': ['kw_avg_min'], 'kw_avg_min': ['kw_max_min'], 'kw_min_max': ['kw_min_avg'], 'kw_max_max': ['kw_min_min'], 'kw_min_avg': ['kw_min_max'], 'kw_max_avg': ['kw_avg_avg'], 'kw_avg_avg': ['kw_max_avg'], 'self_reference_min_shares': ['self_reference_avg_sharess'], 'self_reference_max_shares': ['self_reference_avg_sharess'], 'self_reference_avg_sharess': ['self_reference_max_shares', 'self_reference_min_shares'], 'global_sentiment_polarity': ['rate_positive_words'], 'global_rate_negative_words': ['rate_negative_words'], 'rate_positive_words': ['rate_negative_words', 'global_sentiment_polarity'], 'rate_negative_words': ['rate_positive_words', 'global_rate_negative_words'], 'avg_negative_polarity'

AFTER THIS STUDY, BY HAND WE ANALYIZE THE MOST CORRELATED FEATURES AND WE REMOVE THE ONE THAT ARE LESS IMPORTANT TO OUR TARGET VALUE

This is our result:


toRemove = {'n_non_stop_words', 'n_unique_tokens','kw_max_max','kw_max_min','kw_min_max','kw_max_avg','self_reference_min_shares','self_reference_max_shares',
'global_sentiment_polarity','global_rate_negative_words','avg_negative_polarity','abs_title_sentiment_polarity'}


{'n_tokens_content': ['n_non_stop_words', 'n_unique_tokens'], 'n_unique_tokens': ['n_non_stop_unique_tokens', 'n_tokens_content', 'n_non_stop_words'], 'n_non_stop_words': ['n_tokens_content', 'n_unique_tokens'], 'n_non_stop_unique_tokens': ['n_unique_tokens'], 'kw_min_min': ['kw_max_max'], 'kw_max_min': ['kw_avg_min'], 'kw_avg_min': ['kw_max_min'], 'kw_min_max': ['kw_min_avg'], 'kw_max_max': ['kw_min_min'], 'kw_min_avg': ['kw_min_max'], 'kw_max_avg': ['kw_avg_avg'], 'kw_avg_avg': ['kw_max_avg'], 'self_reference_min_shares': ['self_reference_avg_sharess'], 'self_reference_max_shares': ['self_reference_avg_sharess'], 'self_reference_avg_sharess': ['self_reference_max_shares', 'self_reference_min_shares'], 'global_sentiment_polarity': ['rate_positive_words'], 'global_rate_negative_words': ['rate_negative_words'], 'rate_positive_words': ['rate_negative_words', 'global_sentiment_polarity'], 'rate_negative_words': ['rate_positive_words', 'global_rate_negative_words'], 'avg_negative_polarity': ['min_negative_polarity'], 'min_negative_polarity': ['avg_negative_polarity'], 'title_subjectivity': ['abs_title_sentiment_polarity'], 'abs_title_sentiment_polarity': ['title_subjectivity']}


35 most important features that are more likely to determine target value

kw_avg_avg                    0.259361
kw_max_avg                    0.227842
self_reference_avg_sharess    0.193903
self_reference_min_shares     0.181423
self_reference_max_shares     0.170796
data_channel_world            0.166275
LDA_02                        0.155207
data_channel_lifestyle        0.137921
data_channel_entertainment    0.117396
data_channel_socmed           0.113001
global_subjectivity           0.112726
weekday_saturday              0.109282
kw_min_avg                    0.104967
weekday_sunday                0.097623
data_channel_tech             0.094187
kw_avg_min                    0.093259
kw_max_min                    0.092380
num_hrefs                     0.090480
num_imgs                      0.086005
global_sentiment_polarity     0.081447
num_keywords                  0.077557
n_non_stop_unique_tokens      0.072598
rate_negative_words           0.072180
LDA_01                        0.071839
global_rate_positive_words    0.071263
LDA_03                        0.066014
title_sentiment_polarity      0.059091
kw_min_max                    0.058559
rate_positive_words           0.056733
max_positive_polarity         0.055917
average_token_length          0.054841
avg_positive_polarity         0.053024
LDA_04                        0.050768
weekday_wednesday             0.050451



**preprocessing**

In [23]:
## Initializing the dataset
path_eva = "./data/evaluation.csv"
path_dev = "./data/development.csv"
df = pd.read_csv(path_dev)
df_eval = pd.read_csv(path_eva)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
id_col = df_eval['id']


##  Deleting silly columns
df.drop(['url','id','timedelta'], axis = 1, inplace = True)
df_eval.drop(['url','id','timedelta'], axis = 1, inplace = True)


#   Remove strange outliers with some data with 0 values
mask = (df['average_token_length'] == 0) & (df['n_tokens_content'] == 0)
df.drop(df[mask].index, axis = 0, inplace = True)


##Filling NaN values in num_imgs and num_videos using zero
NaN_columns = ['num_imgs','num_videos']
for el in NaN_columns:
    mean_values = df.groupby('data_channel')[el].mean()
    # Iterate over each group and fill NaN values with the corresponding mean
    for group, mean in mean_values.items():
      df.loc[df['data_channel'] == group, el] = df.loc[df['data_channel'] == group, el].fillna(0)
for el in NaN_columns:
    mean_values_eval = df.groupby('data_channel')[el].mean()
    # Iterate over each group and fill NaN values with the corresponding mean
    for group, mean in mean_values_eval.items():
      df_eval.loc[df_eval['data_channel'] == group, el] = df_eval.loc[df_eval['data_channel'] == group, el].fillna(0)

##  filling NaN values in num_keywords grouping by 'data_channel' and calculate the mean
mean_values = df.groupby('data_channel')["num_keywords"].mean()
for group, mean in mean_values.items():
  df.loc[df['data_channel'] == group, "num_keywords"] = df.loc[df['data_channel'] == group, "num_keywords"].fillna(mean)
  df_eval.loc[df_eval['data_channel'] == group, "num_keywords"] = df_eval.loc[df_eval['data_channel'] == group, "num_keywords"].fillna(mean)

##  Transforming several features into a normal distribution shape using logaritmic transformation
logTransformation = ['n_tokens_content','num_hrefs','num_self_hrefs','num_imgs','num_videos','kw_max_min','kw_avg_min','kw_min_max','kw_min_avg','kw_max_max','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares']
df[logTransformation] = np.log(1.001 + df[logTransformation])
df_eval[logTransformation] = np.log(1.001 + df_eval[logTransformation])



##  Z-score normalization of our data
for el in df.columns:
  if(el != "shares" and el != "data_channel" and el != "weekday"):
    df[el] = (df[el] - df[el].mean()) / df[el].std()

for el in df_eval.columns:
  if(el != "shares" and el != "data_channel" and el != "weekday"):
    df_eval[el] = (df_eval[el] - df[el].mean()) / df[el].std()



##  One-Hot Encoding
df = pd.get_dummies(df, columns=['data_channel','weekday'])
df_eval = pd.get_dummies(df_eval, columns=['data_channel','weekday'])



DROPPING REDUNDANT COLUMNS AND REMOVING OUTLIER MORE LIKELY TO PREDICT TARGET VALUE IN UNIVARIATE WAY

In [24]:
##  Dropping redundant columns using our previous computation using correlation matrix
toRemove = ['n_non_stop_words', 'n_unique_tokens','kw_max_max','kw_max_min','kw_min_max','kw_max_avg','self_reference_min_shares','self_reference_max_shares','global_sentiment_polarity','global_rate_negative_words','avg_negative_polarity','abs_title_sentiment_polarity']
df.drop(toRemove,axis=1,inplace=True)
df_eval.drop(toRemove,axis=1,inplace=True)

##  Removing outlier using Quartile. Values in the interval between Q1 - fact * IQR and Q3 + fact * IQR
removingOutlierColumn('shares',df, fact = 8)
removingOutlierColumn('kw_avg_avg',df,1.5)
removingOutlierColumn("self_reference_avg_sharess",df,4)
removingOutlierColumn('kw_avg_min',df,fact = 5)


Number of outliers in "shares : 825
-13095.0
16740.0
1755.0
Number of outliers in "kw_avg_avg : 673
-1.5920606553447336
1.5979979076153152
0.7975146407400122
Number of outliers in "self_reference_avg_sharess : 1376
-0.8711713696850538
0.5857736571323898
0.16188278075749374
Number of outliers in "kw_avg_min : 635
-2.5225310078949876
2.7971381122145598
0.4836062836463225


HYPERPARAMETER TUNING

In [25]:
y = df["shares"]
df.drop("shares",axis=1,inplace=True)
X = df

# Create the Random Forest Regressor
#rf_regressor = RandomForestRegressor(random_state = 42)
rf_regressor = GradientBoostingRegressor()

# Define the hyperparameters to tune
parameters = {
     'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}
parameters = {
    'n_estimators': [100, 200, 300],  # Number of boosting stages
    'learning_rate': [0.1, 0.05, 0.01],  # Learning rate
    'max_depth': [3, 4, 5],  # Maximum depth of each tree
}

# Create the RMSE scorer
scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)

# Perform grid search with cross-validation
grid_search = GridSearchCV(rf_regressor, parameters, scoring=scorer, cv=5)
grid_search.fit(X, y)  # X_train and y_train are your training data

# Print the best hyperparameters and RMSE score
print("Best Hyperparameters: ", grid_search.best_params_)
print("Best RMSE Score: ", (-grid_search.best_score_))

# Get the best model
best_rf_model = grid_search.best_estimator_

print(best_rf_model)

EVALUATION ON TRAIN SET

In [ ]:
y = df["shares"]

df.drop("shares",axis=1,inplace=True)
X = df


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_regressor = RandomForestRegressor(300,max_depth=35,random_state=33)
rf_regressor.fit(X_train, y_train)

# Predict on the testing data
y_pred = rf_regressor.predict(X_test)
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", rmse)
df["shares"] = y

EVALUATION ON VAL SET

In [ ]:
y_train = df["shares"]
df.drop("shares",axis=1, inplace=True)
X_train = df
X_test = df_eval


##  Initialization the model with best parameters found in training
rf_regressor = RandomForestRegressor(n_estimators = 300, max_depth = 35, random_state = 42)

rf_regressor.fit(X_train,y_train)

y_pred = rf_regressor.predict(X_test)

df["shares"] = y_train

##  Creating CSV file for submitting the results
csv_df = pd.DataFrame(columns=['Id', 'Predicted'])
csv_df['Id'] = id_col
csv_df['Predicted'] = y_pred
csv_df.to_csv('./output.csv', columns=['Id','Predicted'], index=False)